# Multi-Layer Perception

In this lesson, we're going to be exploring how to create our very own neural network - from scratch!


In [ ]:
# If in Colab, uncomment the following line
# !pip install -q gradio

In [1]:
import os
import unittest
from typing import List

import numpy as np
import pandas as pd
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import gradio as gr
import matplotlib.pyplot as plt
import plotly.express as px

from torch.utils.data import DataLoader

## Part 1: What is a Neural Network?

You might've heard a lot about neural networks and deep learning already, but what exactly are they?

![neural network](images/Neural-Networks-Architecture.png)

A **neural network** is a machine learning model that is inspired by the human brain. It is composed of a large number of highly interconnected units called **neurons** working in unison to solve specific problems. Neural networks are particularly good at solving problems that are too complex for many traditional machine learning algorithms.

While we're talking about the human brain, it's important to emphasize that artificial neural networks are *vastly* different from how the human brain actually operates. The human brain is much more complex and powerful than any artificial neural network that we have today. In fact, we don't even know how the human brain works yet. We're still trying to figure that out.

That being said, let's take a step back and look at how a biological neuron works. A neuron is a cell that receives signals from its dendrites, processes these signals, and then sends signals to its axon. The axon then passes the signal to the dendrites of other neurons. This is how information is passed from one neuron to another in the brain.

![biological neuron](images/neuron_anatomy.jpg)

Similarly, a neuron in a neural network will also receive input signals and send output signals to other neurons.

![neural network neuron](images/nn_neuron.png)

If this is your first time seeing an artificial neuron, you might be wondering why it looks so complicated. Don't worry, we'll break it down into smaller pieces and explain each part in detail.

All the $x$'s represent an **input** to a neuron. This might be something like the pixels of an image, the words of a sentence, or the features of a data point.

The $w$'s represent the **weights** of the neuron, which are constants that are multiplied to all the inputs. The weights are the parameters of the neuron that are learned during training.

You can see that the first step of the neuron is to multiply each input by its corresponding weight and then add all the results together. This is called the **weighted sum** of the inputs.

The $b$ represents the **bias** of the neuron. The bias is also a parameter of the neuron that is learned during training. Again, it's really just a constant, where this time it's added to the weighted sum of the inputs.

The $f$ represents the **activation function** of the neuron. The activation function is a nonlinear function (more on the nonlinear part later) that takes in the weighted sum of the inputs and bias and outputs some new value. Now you can see that the neuron is just a function that takes in some inputs, does some multiplying and adding, and outputs some value.

Now that we have a pretty good idea of how a single neuron works, let's code up a simple neuron in Python.

In [7]:
class SimpleNeuron():
    def __init__(self, weights, bias, activation_function):
        self.weights = weights
        self.bias = bias
        self.activation_function = activation_function

    def forward(self, input):
        assert len(input) == len(self.weights)
        self.input = input
        weighted_sum = sum([self.weights[i] * self.input[i] for i in range(len(self.input))])
        self.output = self.activation_function(weighted_sum + self.bias)
        # Alternative solution without list comprehension
        # outputs = []
        # for i in range(len(self.input)):
        #     self.output.append(self.weights[i] * self.input[i])
        # weighted_sum = sum(outputs)
        # self.output = self.activation_function(weighted_sum + self.bias)
        return self.output

def activation_function(x):
    """
    Write an activation function that returns 0 if x is less than 0, otherwise returns x.
    """
    return x if x > 0 else 0

In [ ]:
neuron = SimpleNeuron([0.5, -0.5, 1], 1, activation_function)
neuron.forward([0.35, 0.40, 0.9])

![honest_work_meme](images/honest_work_meme.jpg)

So now that we have finished writing our neuron, we're going to move up one level of abstraction and talk about how neurons make a neural network layer. A **layer** is a collection of neurons that are all connected to the same inputs. Specifically, we are implementing a **fully-connected layer**, which means that every neuron in the layer is connected to every input.

![layer](images/nn_layer.png)

An example of a fully-connected layer with four neurons for three inputs.

Layers are useful for a few reasons. For one, if we want to have the neural network take in a bunch of inputs and output a single number, we should create a layer with a single neuron. If we want to have the neural network take in a bunch of inputs and output a bunch of numbers (like predicting the probability that a specific object is inside the image), we should instead create a layer with multiple neurons.

Another benefit for a layer abstraction that we can concatenate more and more layers to create a **deep neural network**, which will allow for neural networks to learn more complex behaviors like detecting an object in an image or translating a sentence from one language to another.

Below we'll implement a layer using our `SimpleNeuron` code.

In [ ]:
class SimpleLayer:
    def __init__(self, neurons: List[SimpleNeuron]):
        self.neurons = neurons

    def forward(self, inputs):
        return [neuron.forward(inputs) for neuron in self.neurons]

## Part 1.5: Rewriting with Linear Algebra

### Matrix Multiplication

To be completely honest, the previous section was a little misleading. Although it may be more intuitive to use for loops and if statements to write our neural network methods, it's actually not the best way to implement them.

![surprised_pikachu](images/surprised_pikachu_meme.png)

In reality, neural networks are internally represented with matrices. The reason why this is so important is because matrix multiplication can be parallelized and greatly sped up using GPU acceleration hardware. This is absolutely necessary when we're using neural networks for enormous amounts of high-dimensional data, where we're sometimes processing millions of numbers at a time.

Our previous implementation requires instantiating a neuron for every neuron that we want in our layer, and then we would iterate through all the weights of that neuron, which we would then do for every neuron in the next layer. To rewrite this, we can recognize the fact that every neuron in a layer is doing the same thing - some sort of weighted sum of the inputs, followed by an activation function. Weighted sums are linear operations, so we can just represent the entire layer (aside from the activation function) as a matrix.

![neural_network_matrix](images/nn_matrix.png)

After representing the weights as a matrix, we can perform a matrix multiplication with the input vector to get the output vector. This will be much faster than iterating through each neuron individually and will allow for GPU parallelization. You can also visually see how the weighted sums are exactly the same as the matrix multiplication shown, which shows how the vast majority of the computation is just matrix multiplication.

![matrix_multiplication](images/nn_matrix_colored.png)

The examples we've been doing so far shows doing matrix multiplication with an weight matrix and an input vector, but we can actually extend this and calculate a **batch** of input vectors, or a matrix of input vectors. This is useful because we can then do matrix multiplication with a batch of input vectors and a weight matrix to get a batch of output vectors. Doing this batch multiplication will allow us to get more stable gradients and will allow us to parallelize the matrix multiplication even more.

![batch_matrix_multiplication](images/nn_matrix_batch.jpg)

### Nonlinear Activation Functions

We also very briefly talked about how the activation functions are *nonlinear* functions that are applied to the output of the linear transformation. This is really important because we want to be able to model highly nonlinear relationships, like matching a face to a name, or predicting the price of a house based on its features.

Suppose that we didn't have these nonlinear activation functions. Then, the output of the linear transformation would just be a linear combination of the input features. This means that no matter how many layers we have, the output of the last layer would still be a linear combination of the input features. This is one of the motivating factors behind the use of nonlinear activation functions.

A common activation function that you'll often see (and what we implemented earlier) is the Rectified Linear Unit (ReLU) function. This function is defined as:

$$ f(x) = \max(0, x) $$

which essentially means that if the input is negative, the output is 0, and if the input is positive, the output is the input. This is a very simple function, but it is very effective in practice. It is also very easy to compute, which is a plus.

Now let's see if we can rewrite our `SimpleLayer` class with matrix multiplication.

In [13]:
class UpdatedLayer:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size)
        self.biases = np.zeros(output_size)
    
    def initialize_params(self, weights, biases):
        self.weights = weights
        self.biases = biases
    
    def activation_function(self, input):
        return np.maximum(input, 0)

    def forward(self, input):
        return self.activation_function(self.weights.T @ input + self.biases)

## Part 2: Optimizing the Neural Network

We almost have all of our pieces together for creating our very own neural network! The last thing we need to do is define our loss function and implement backpropagation.

### Loss Function

The **loss function** is a function that takes in the output of the neural network and the true label, and outputs a scalar value that represents how well the neural network is performing. The loss function is also known as the **cost function** or the **error function**. The loss function is the function that we are trying to minimize, so that the neural network can learn to make better predictions. We will be using mean-squared error, or MSE, as our loss function, which is defined as:

$$MSE = \frac{1}{n} \sum_{i=1}^{n} (y_{i} - \hat{y}_{i})^{2}$$

where $n$ is the number of samples, $y_{i}$ is the true label, and $\hat{y}_{i}$ is the predicted label. We will be using MSE because it is a pretty simple and common loss function, and it is easy to calculate the gradient of MSE with respect to the weights (more on gradients and whatnot below).

### Backpropagation

This entire time, we have been designing the **forward pass** of our neural network, which is the process of taking in an input and passing it through the network to get an output. Now, we will be implementing the **backward pass**, which is the process of taking the output of the forward pass and using it to update the weights of the network. This is also known as **backpropagation**.

![backpropagation](images/backpropagation.jpeg)

In order for the neural network to "learn", it needs to follow the gradient of the loss function with respect to the weights. Once we have the gradient, we can update the weights by subtracting the gradient from the weights. This is known as **gradient descent**, described by the formula:

$$\theta_{i} = \theta_{i-1} - \alpha \frac{\partial L}{\partial \theta}$$

where $\theta_{i}$ is the weights at the current iteration, $\theta_{i-1}$ is the weights at the previous iteration, $\alpha$ is the learning rate, and $L$ is the loss function.

Alright, easy enough - but how do we get the gradient in the first place? This is where backpropagation comes in. Backpropagation is the process of calculating the gradient of the loss function with respect to the weights. We will be using the chain rule to calculate the gradient. Recall that the chain rule states that:

$$\frac{\partial L}{\partial \theta} = \frac{\partial L}{\partial \hat{y}} \frac{\partial \hat{y}}{\partial \theta}$$

where $\hat{y}$ is the output of the forward pass. We can see that we can calculate the gradient by calculating the gradient of the loss function with respect to the output of the forward pass, and then multiplying it by the gradient of the output of the forward pass with respect to the weights. Therefore, we will always be able to derive the change that the weights needs to see in order to minimize the loss function.

![gradient_descent](images/gradient_descent.gif)

## Part 3: Creating an MLP for Classification

We mentioned earlier that we'll be writing our very own classifier **Multi-Layer Perceptron**, or **MLP**, from scratch. An MLP is the most basic form of a neural network, which consists of multiple fully-connected layers like we've been dealing with so far.

Suppose you're in class taking notes in your notebook, and you really want to convert your handwritten notes into really neat, typed notes. You could use a neural network to do this! You could take a picture of your handwritten notes, and feed it into a neural network that will output the typed notes. This is called **optical character recognition**, or **OCR**. We'll be using OCR to classify handwritten digits.

You might be wondering, our neural networks so far have been able to give outputs as numbers, but we're going to be classifying images into different categories. How do we do that? Well, we'll be using a technique called **one-hot encoding**. One-hot encoding is a way to represent categorical data in a way that a neural network can understand. For example, if we have a dataset of images of cats, dogs, and birds, we can represent the labels as follows:

| Image | Label |
| --- | --- |
| cat | 100 |
| dog | 010 |
| bird | 001 |

In our case, we will be using the MNIST dataset, which consists of images of handwritten digits. The labels are the numbers 0-9, so we can represent them as follows:

| Image | Label |
| --- | --- |
| 0 | 1000000000 |
| 1 | 0100000000 |
| 2 | 0010000000 |
| 3 | 0001000000 |
| 4 | 0000100000 |
| 5 | 0000010000 |
| 6 | 0000001000 |
| 7 | 0000000100 |
| 8 | 0000000010 |
| 9 | 0000000001 |

Let's go ahead and take a look at what the MNIST dataset looks like.

In [ ]:
im_size = 24

train_dataset = datasets.MNIST(
    root='dataset/',
    train=True,
    transform=transforms.Compose([
        transforms.RandomCrop(im_size),
        transforms.ToTensor()
    ]),
    download=True)
test_dataset = datasets.MNIST(
    root='dataset/',
    train=False,
    transform=transforms.Compose([
        transforms.RandomCrop(im_size),
        transforms.ToTensor()
    ]),
    download=True)

Let's quickly take a look at exactly how many images that we will be using for our dataset. Take a guess - maybe on the order of hundreds? Thousands?

In [ ]:
train_dataset, test_dataset

You can see that our datasets actually consist of tens of thousands of images! This shows a major caveat for using neural networks - a lot of the time, neural networks require large amounts of data to train on so they can perform well and generalize properly.

Now, this isn't to say that neural networks are useless if you don't have a lot of data; there are definitely ways to squeeze out performance from minimal datasets. However, for the purposes of this lesson, we will be using a dataset that has a lot of data so we can see the full power of neural networks.

In [ ]:
rand_int = np.random.choice(len(train_dataset))
plt.imshow(train_dataset[rand_int][0][0], cmap='Greys_r')
print(f"Label: {train_dataset[rand_int][1]}")
print(f"Shape: {train_dataset[rand_int][0].shape}")
plt.show()

So now we have all of the pieces we need to create our MLP model from scratch!

In [5]:
class MLP:
    """
    A simple implementation of a Multi-Layer Perceptron (MLP) with one hidden layer.
    """

    def __init__(self, feature_size, hidden_size, num_classes, lr=0.01):
        """
        Initialize the MLP with random weights and biases,
        as well as the hyperparameters.

        Parameters:
            feature_size (int): The number of features in the input data.
            hidden_size (int): The number of hidden units in each hidden layer.
            num_classes (int): The number of classes in the output.
            lr (float): The learning rate.
        """
        # X is the input data
        self.X = None

        # y_hat is the output of the MLP, and y is the true label
        self.y_hat = None
        self.y = None

        # W_i and b_i are the weights and biases of the MLP for the ith layer, respectively
        # z_i is the output of the linear layer before the activation function
        # o_i is the output of the linear layer after the activation function
        stdev1 = 1 / np.sqrt(feature_size)
        self.W1 = np.random.uniform(low=-stdev1, high=stdev1, size=(feature_size, hidden_size))
        self.b1 = np.random.uniform(low=-stdev1, high=stdev1, size=(hidden_size))
        self.z1 = None
        self.o1 = None

        stdev2 = 1 / np.sqrt(hidden_size)
        self.W2 = np.random.uniform(low=-stdev2, high=stdev2, size=(hidden_size, num_classes))
        self.b2 = np.random.uniform(low=-stdev2, high=stdev2, size=(num_classes))
        # We do not need to specify z2 or o2, since this is the output of the MLP (y_hat)

        # The learning rate
        self.lr = lr
        self.num_classes = num_classes

    def relu(self, X):
        """
        The ReLU activation function.

        Parameters:
            X (np.array): The input data of shape (N, D) where N is the number of samples and D is the number of features.
        Returns:
            np.array: The output of the ReLU activation function of shape (N, D).
        """
        return np.maximum(0, X)

    def softmax(self, X):
        """
        The softmax activation function. We won't be using this function inside our neural network for
        the sake of simplicity but we will still implement it to convert our output to approximate probabilities.
        Keep in mind that it is very common to use the softmax function as the activation function for the output layer.

        Parameters:
            X (np.array): The input data of shape (N, D) where N is the number of samples and D is the number of features.
        Returns:
            np.array: The output of the softmax activation function of shape (N, D).
        """

        """
        TEACHER NOTE: This implementation is a more numerically stable version of the softmax function
        by subtracting the maximum value of each row from each row to prevent overflow.
        See: https://ogunlao.github.io/2020/04/26/you_dont_really_know_softmax.html#numerical-stability-of-softmax
        """

        # Subtract the maximum value of each row from each row to prevent overflow
        X = X - np.max(X, axis=1, keepdims=True)
        return np.exp(X) / np.sum(np.exp(X), axis=1, keepdims=True)

    def one_hot(self, y, num_classes):
        """
        Convert the true labels to one-hot encoded vectors.

        Parameters:
            y (np.array): The true labels of shape (N,).
            num_classes (int): The number of classes in the output.
        Returns:
            np.array: The one-hot encoded vectors of shape (N, num_classes).
        """

        """
        TEACHER NOTE: Admittedly, this method (and a few others) are redundant since
        the MLP will store these values as attributes. However, I wanted to make the
        code more readable and explicit while maintaining a more functional programming style.
        """
        one_hot = np.zeros((len(y), num_classes))
        one_hot[np.arange(len(y)), y.squeeze()] = 1
        return one_hot

    def forward(self, X, y):
        """
        Forward pass of the MLP.

        Parameters:
            X (np.array): The input data of shape (N, D) where N is the number of samples and D is the number of features.
            y (np.array): The true labels of shape (N,).
        Returns:
            self.y_hat (np.array): The output of the MLP of shape (N,).
        """
        # Set the input data and true labels
        X = X.reshape(-1, self.W1.shape[0])
        self.X = X
        self.y = y

        # Perform the forward pass and set the intermediate output values
        self.z1 = X @ self.W1 + self.b1
        self.o1 = self.relu(self.z1)
        self.y_hat = self.o1 @ self.W2 + self.b2
        return self.y_hat

    def mse_loss(self, y_hat, y):
        """
        The mean squared error loss function.

        Parameters:
            y_hat (np.array): The output of the MLP of shape (N,).
            y (np.array): The true labels of shape (N,).
        Returns:
            float: The mean squared error loss.
        """
        return np.mean((y_hat - y) ** 2)

    def linear_gradient(self, input, weights, dL_doutput):
        """
        Calculate the gradient of the loss w.r.t. the input, weights, and biases of a linear layer.
        Function inspired by fast.ai implementation: https://www.youtube.com/watch?v=vGdB4eI4KBs

        Parameters:
            input (np.array): The input data of shape (N, D) where N is the number of samples and D is the number of features.
            weights (np.array): The weights of the linear layer of shape (D, H) where H is the number of hidden units.
            dL_doutput (np.array): The gradient of the loss w.r.t. the output of the linear layer of shape (N, H).
        Returns:
            dL_dinput (np.array): The gradient of the loss w.r.t. the input of shape (N, D).
            dL_dweights (np.array): The gradient of the loss w.r.t. the weights of shape (D, H).
            dL_dbias (np.array): The gradient of the loss w.r.t. the biases of shape (H,).
        """

        """
        TEACHER NOTE: One can derive the gradient of the loss w.r.t. the weights and biases
        for a linear layer by using the chain rule:
        dL_dW = dL_doutput * doutput_dW, where doutput_dW = input
        dL_db = dL_doutput * doutput_db, where doutput_db = 1
        dL_dinput = dL_doutput * doutput_dinput, where doutput_dinput = weights

        Keep in mind that the shapes of the gradients are the same as the shapes of the weights and biases.
        Therefore, we need to use the transpose of the weights to get the correct shapes.
        """
        dL_dinput = dL_doutput @ weights.T
        dL_dweights = np.sum(
            np.expand_dims(input, -1) * np.expand_dims(dL_doutput, 1), axis=0
        )
        dL_dbias = np.sum(dL_doutput, axis=0)

        return dL_dinput, dL_dweights, dL_dbias

    def backpropagate(self):
        """
        Backpropagation of the MLP using gradient descent.
        Parameters:
            y (np.array): The true labels of shape (N,).
        Returns:
            dL_dW1 (np.array): The gradient of the loss w.r.t. the weights of the first hidden layer of shape (D, H).
            dL_db1 (np.array): The gradient of the loss w.r.t. the biases of the first hidden layer of shape (H,).
            dL_dW2 (np.array): The gradient of the loss w.r.t. the weights of the second hidden layer of shape (H,).
            dL_db2 (np.array): The gradient of the loss w.r.t. the biases of the second hidden layer of shape (1,).
        """

        # Calculate the gradient of the loss w.r.t. y_hat
        dL_dyhat = 2 * (self.y_hat - self.one_hot(self.y, self.num_classes)) / self.y_hat.size

        # Backpropagate and update the weights and biases of the second hidden layer
        dL_do1, dL_dW2, dL_db2 = self.linear_gradient(self.o1, self.W2, dL_dyhat)
        self.W2 -= self.lr * dL_dW2
        self.b2 -= self.lr * dL_db2

        # Calculate the gradient of o1 w.r.t. z1 (the output before the activation function)
        do1_dz1 = (self.o1 > 0).astype(int)

        # Backpropagate and update the weights and biases of the first hidden layer
        dL_dX, dL_dW1, dL_db1 = self.linear_gradient(self.X, self.W1, dL_do1 * do1_dz1)
        self.W1 -= self.lr * dL_dW1
        self.b1 -= self.lr * dL_db1

        return dL_dW1, dL_db1, dL_dW2, dL_db2

### Unit Tests

A **unit test** is a small piece of code that tests a very specific piece of functionality. Unit tests are used to test individual units of code, such as individual functions, methods, or classes. Realistically, a software developer will rarely be able to catch bugs simply by looking through their code (or be able to have perfectly working code on the first try). Unit tests are a great way to automatically test your code to find bugs early.

Below we included some unit tests to confirm that your code is working as expected. Over time, less unit tests will be outright given and you'll have to write your own comprehensive unit tests to ensure that all of your functions and classes work as intended.

In [58]:
# Unit tests to validate the MLP code
class TestMLP(unittest.TestCase):
    def setUp(self):
        self.mlp = MLP(2, 3, 2, lr=0.1)
        # Manually assign the weights and biases to test the forward pass
        self.W1 = np.array([[0.1, 0.2, 0.3], [-0.1, -0.3, 0.4]])
        self.b1 = np.array([0.5, 0.6, 0.7])
        self.W2 = np.array([[0.7, 0.7], [0.8, 0.8], [0.9, 0.9]])
        self.b2 = np.array([1.1, 1.2])

        # Manually assign the input and output to test the forward pass
        self.mlp.X = np.array([[0, 1], [1, 0], [1, 1], [0, 0]])
        self.mlp.y = np.array([[0], [1], [1], [0]])

        self.reset_mlp()
        true_vals = self.use_pytorch(self.mlp.X, self.mlp.y, self.W1.T, self.b1, self.W2.T, self.b2)
        self.gt_y_hat = true_vals[0].detach().numpy()
        self.gt_loss = true_vals[1].detach().numpy()
        self.gt_dL_dW1 = true_vals[2].detach().numpy()
        self.gt_dL_db1 = true_vals[3].detach().numpy()
        self.gt_dL_dW2 = true_vals[4].detach().numpy()
        self.gt_dL_db2 = true_vals[5].detach().numpy()

    def reset_mlp(self):
        self.mlp.W1 = self.W1.copy()
        self.mlp.b1 = self.b1.copy()
        self.mlp.W2 = self.W2.copy()
        self.mlp.b2 = self.b2.copy()

    def use_pytorch(self, X, y, W1, b1, W2, b2):
        # Use PyTorch to check forward and backward pass
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y)
        W1 = torch.tensor(W1, dtype=torch.float32, requires_grad=True)
        b1 = torch.tensor(b1, dtype=torch.float32, requires_grad=True)
        W2 = torch.tensor(W2, dtype=torch.float32, requires_grad=True)
        b2 = torch.tensor(b2, dtype=torch.float32, requires_grad=True)

        x = F.linear(X, W1, b1).relu()
        y_hat = F.linear(x, W2, b2)
        loss = F.mse_loss(y_hat, F.one_hot(y, num_classes=2).squeeze().type(torch.float32))
        loss.backward()
        return y_hat, loss, W1.grad, b1.grad, W2.grad, b2.grad

    def test_relu(self):
        self.assertEqual(self.mlp.relu(0), 0)
        self.assertEqual(self.mlp.relu(1), 1)
        self.assertEqual(self.mlp.relu(-1), 0)

    def test_one_hot(self):
        y = np.array([0, 1, 2, 3])
        one_hot = self.mlp.one_hot(y, 4)
        self.assertEqual(one_hot.shape, (4, 4))
        np.testing.assert_allclose(one_hot, np.eye(4))

    def test_forward(self):
        y_hat = self.mlp.forward(self.mlp.X, self.mlp.y)
        self.assertEqual(y_hat.shape, (4,2))
        self.assertTrue(np.allclose(y_hat, self.gt_y_hat))

    def test_mse_loss_1(self):
        y = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]])
        y_hat = np.array([[0.5, 0.5, 0], [0, 0, 1], [1, -1, 0]])
        loss = np.array([self.mlp.mse_loss(y_hat, y)])
        gt_loss = F.mse_loss(torch.Tensor(y_hat), torch.Tensor(y)).detach().numpy()
        np.testing.assert_allclose(loss, gt_loss)

    def test_mse_loss_2(self):
        y_hat = self.mlp.forward(self.mlp.X, self.mlp.y)
        loss = self.mlp.mse_loss(y_hat, self.mlp.one_hot(self.mlp.y, 2))
        np.testing.assert_allclose(loss, self.gt_loss)

    def test_backpropagate(self):
        _ = self.mlp.forward(self.mlp.X, self.mlp.y)
        dL_dW1, dL_db1, dL_dW2, dL_db2 = self.mlp.backpropagate()

        # Check the gradients.
        np.testing.assert_allclose(dL_dW1, self.gt_dL_dW1.T, atol=1e-5)
        np.testing.assert_allclose(dL_db1, self.gt_dL_db1, atol=1e-5)
        np.testing.assert_allclose(dL_dW2, self.gt_dL_dW2.T, atol=1e-5)
        np.testing.assert_allclose(dL_db2, self.gt_dL_db2, atol=1e-5)

        # Check the newly updated parameters.
        np.testing.assert_allclose(self.mlp.W1, self.W1 - self.mlp.lr * self.gt_dL_dW1.T, atol=1e-5)
        np.testing.assert_allclose(self.mlp.b1, self.b1 - self.mlp.lr * self.gt_dL_db1, atol=1e-5)
        np.testing.assert_allclose(self.mlp.W2, self.W2 - self.mlp.lr * self.gt_dL_dW2.T, atol=1e-5)
        np.testing.assert_allclose(self.mlp.b2, self.b2 - self.mlp.lr * self.gt_dL_db2, atol=1e-5)
    
    def test_train_loop(self):
        X = self.mlp.X.copy()
        y = self.mlp.y.copy()
        W1 = self.W1.copy()
        b1 = self.b1.copy()
        W2 = self.W2.copy()
        b2 = self.b2.copy()
        for _ in range(5):
            self.mlp.forward(self.mlp.X, self.mlp.y)
            self.mlp.backpropagate()
            gt_vals = self.use_pytorch(X, y, W1.T, b1, W2.T, b2)
            W1 -= self.mlp.lr * gt_vals[2].detach().numpy().T
            b1 -= self.mlp.lr * gt_vals[3].detach().numpy()
            W2 -= self.mlp.lr * gt_vals[4].detach().numpy().T
            b2 -= self.mlp.lr * gt_vals[5].detach().numpy()

        # Check the newly updated parameters.
        np.testing.assert_allclose(self.mlp.W1, W1, atol=1e-5)
        np.testing.assert_allclose(self.mlp.b1, b1, atol=1e-5)
        np.testing.assert_allclose(self.mlp.W2, W2, atol=1e-5)
        np.testing.assert_allclose(self.mlp.b2, b2, atol=1e-5)

loader = unittest.TestLoader()
suite = unittest.TestSuite()
suite.addTests(loader.loadTestsFromTestCase(TestMLP))
unittest.TextTestRunner(verbosity=3).run(suite)

test_backpropagate (__main__.TestMLP) ... 

ok
test_forward (__main__.TestMLP) ... ok
test_mse_loss_1 (__main__.TestMLP) ... ok
test_mse_loss_2 (__main__.TestMLP) ... ok
test_one_hot (__main__.TestMLP) ... ok
test_relu (__main__.TestMLP) ... ok
test_train_loop (__main__.TestMLP) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.065s

OK


<unittest.runner.TextTestResult run=7 errors=0 failures=0>

## Part 4: Training the MLP Model

If you have completed the previous parts, you should have a working MLP model. Just for a second, I want you to take a step back and really reflect on the awesome work you've so far, which is create a neural network *completely from scratch*. You've created a model that can learn from data and make predictions. That's pretty cool!

Unfortunately, our job is not quite done yet. Although we've finished designing the infrastructure, we still need to train the model.

The general infrastructure of training a model is as follows:

```
1.  For each epoch:
    1.  For each training example:
        1.  Forward pass
        2.  Compute loss
        3.  Backward pass and update parameters
    2.  For each testing example:
        1.  Forward pass
        2.  Compute loss
        3.  Compute testing accuracy (optional)
```

In [130]:
def train_mlp(mlp, train_loader, test_loader, epochs=10, train_losses=[], test_losses=[]):

    test_total = 0
    test_num_correct = 0

    for epoch in range(1, epochs+1):

        # Reset the loss for each epoch.
        train_loss = 0.0
        test_loss = 0.0

        # Reset the total and correct test-time predictions.
        test_total = 0
        test_num_correct = 0

        # Iterate through the training dataset.
        for data in train_loader:

            # Obtain the inputs and labels.
            X, y = data
            X = X.numpy()
            y = y.numpy()

            # Perform the forward pass.
            one_hot_y = mlp.one_hot(y, mlp.num_classes)
            y_hat = mlp.forward(X, y)

            # Compute the loss and update the gradients.
            loss = mlp.mse_loss(y_hat, one_hot_y)
            mlp.backpropagate()

            # Update the running loss.
            train_loss += loss

        print(f"Epoch {epoch} training loss: {train_loss / len(train_loader)}")

        # Iterate through the testing dataset.
        for  data in test_loader:

            # Obtain the inputs and labels.
            X, y = data
            X = X.numpy()
            y = y.numpy()

            # Perform the forward pass.
            one_hot_y = mlp.one_hot(y, mlp.num_classes)
            y_hat = mlp.forward(X, y)

            # Update the total and correct predictions in the testing dataset.
            test_total += y.shape[0]
            predicted_y = np.argmax(y_hat, axis=1)
            # test_num_correct += np.all((predicted_y == one_hot_y), axis=1).sum()
            test_num_correct += np.sum(predicted_y == y)

            loss = mlp.mse_loss(y_hat, one_hot_y)
            test_loss += loss

        print(f"Epoch {epoch} testing loss: {test_loss / len(test_loader)}")
        print(f"Testing accuracy: {test_num_correct / test_total}")
        train_losses.append(train_loss / len(train_loader))
        test_losses.append(test_loss / len(test_loader))

    print("Training finished.")

After you have finished writing your training loop, you can run the following cell to train your model.

You can see that there are some variables that you can adjust. These are called **hyperparameters** and these are the variables that you can change to affect the training process. What machine learning practioners do is that they try different values for these hyperparameters and see which one works best. To briefly discuss each hyperparameter:
- The **learning rate** is the size of the step that the optimizer takes to update the weights. If the learning rate is too small, the training process will take a long time. If the learning rate is too large, the training process will be unstable and the model will not converge.
- The **batch size** is the number of samples that the model will use to update the weights. If the batch size is too small, the training process will be unstable and the model will not converge. If the batch size is too large, the training process will take a long time.
- The **number of epochs** is the number of times that the model will see the entire training dataset. If the number of epochs is too small, the model will not be able to learn the patterns in the data. If the number of epochs is too large, we will have to wait a long time for the model to finish training.

Go ahead and adjust the hyperparameters to see how they affect the training process.

In [ ]:
# Adjust the values of the hyperparameters to obtain better accuracy.
learning_rate = 0.5
batch_size = 1024
epochs = 20

mlp = MLP(feature_size=im_size**2, hidden_size=10, num_classes=10, lr=learning_rate)

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

train_losses, test_losses = [], []
train_mlp(mlp, train_dataloader, test_dataloader, epochs=epochs, train_losses=train_losses, test_losses=test_losses)

We can see how well the model training went by plotting the loss over time. Ideally, we should be able to see the loss decreasing over the number of epochs.

In [ ]:
def plot_losses(train_losses, test_losses):
    epochs = len(train_losses)

    # Create a DataFrame for the losses
    data = {
        'Epoch': list(range(1, epochs + 1)),
        'Train Loss': train_losses,
        'Test Loss': test_losses
    }
    df_losses = pd.DataFrame(data)

    # Create the line plot
    fig = px.line(df_losses, x='Epoch', y=['Train Loss', 'Test Loss'],
                  labels={'value': 'Loss'})

    # Customize the plot
    fig.update_layout(title='Training and Test Loss',
                      xaxis_title='Epoch',
                      yaxis_title='Loss',
                      legend_title='Loss Type')

    # Show the plot
    fig.show()

plot_losses(train_losses, test_losses)

In [124]:
# If you're happy with the results, save the weights and biases to prevent having to retrain the model.
if not os.path.exists('mlp_weights/'):
    os.makedirs('mlp_weights/')
np.save('mlp_weights/W1.npy', mlp.W1)
np.save('mlp_weights/b1.npy', mlp.b1)
np.save('mlp_weights/W2.npy', mlp.W2)
np.save('mlp_weights/b2.npy', mlp.b2)

In [7]:
# To load the weights of a previously trained model, you can set `load_weights = True` and run this cell.
load_weights = True

if load_weights:
    mlp = MLP(feature_size=im_size**2, hidden_size=10, num_classes=10)
    mlp.W1 = np.load('mlp_weights/W1.npy')
    mlp.b1 = np.load('mlp_weights/b1.npy')
    mlp.W2 = np.load('mlp_weights/W2.npy')
    mlp.b2 = np.load('mlp_weights/b2.npy')

## Part 5: Testing out your MLP

You should expect your MLP to have a test accuracy of around 75%. This is not great result, but it is a good start especially for the most basic form of a neural network. We will next learn about how we can obtain even better results by using more advanced neural network architectures.

In the meantime, test your handwritten digit classifier on your own handwritten digits! You can use the cell below to draw a digit and see what your MLP thinks it is. Congratulations on building your first neural network!

In [ ]:
def image_classifier(input):
    if input is not None:
        x = input.reshape(-1, mlp.W1.shape[0]) / 255.
        probabilities = mlp.softmax(mlp.forward(x, None))
        values, indices = torch.topk(torch.Tensor(probabilities), 5)
        confidences = {i.item(): v.item() for i, v in zip(indices.squeeze(), values.squeeze())}
        return confidences


gr.Interface(fn=image_classifier,
             inputs=gr.Image(image_mode="L", source="canvas", shape=(im_size, im_size), invert_colors=True, interactive=True),
             outputs="label",
             live=True,
             allow_flagging="never").launch()


Sources used for images:

- https://www.geeksforgeeks.org/artificial-neural-networks-and-its-applications/
- https://askabiologist.asu.edu/neuron-anatomy
- [kratzert.github.io](https://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html)
- https://blog.paperspace.com/intro-to-optimization-in-deep-learning-gradient-descent/